#### **SAMUELE PINO's** *A\* Algorithm*

In [18]:
import logging
import random

def problem(N, seed=42):
    """Creates an instance of the problem"""

    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [19]:
class State():
    """Class for states in A* alghoritm"""

    def __init__(self, state, number_found, g, h, remaining_list = []):
        self.g = g;
        self.h = h;
        self.f = g + h;

        self.remaining_list = remaining_list
        self.state = state
        self.number_found = number_found

#Actual Cost
def g(solution, el):
    return sum(len(element) for element in solution) + len(el)

#Euristic Cost
def h(N, el, number_found):
    return (N - (len(set(el)| number_found)))

def a_star(N):

    n_of_visited_nodes = 0
    start_problem = problem(N)
    goal = set(range(N))
    state_list = []

    open_states = []
    
    print(f"Lists {len(start_problem)}")
    # Create a list of states
    for ind, element in enumerate(start_problem):
        n_of_visited_nodes += 1
        state_list.append(element)
        temp_state = State(state_list, set(element) ,g(state_list, element), h(N,element,set(element)))
        #print(element, temp_state.g, temp_state.h, temp_state.f)
        open_states.append(temp_state)
        state_list = []

    while True:
    
        # consider first state
        ind = 0
        current_state = open_states[ind]

        # For each element in opened_states check if the function has less cost
        # take state with less cost
        # save index
        for indice, open_state in enumerate(open_states):
            if open_state.f < current_state.f:
                current_state = open_state
                ind = indice
        
        # Extract best state
        open_states.pop(ind)
    
        # Get SET of elements covered by current_state
        number_found = current_state.number_found      

        # Check solution correctness (if there is a sol with just one list(?))
        if number_found >= goal:
            print("solution :", current_state.state)
            print("W: ", sum(len(element) for element in current_state.state))
            return

        # If no sol found, save list of current state
        curr_state = current_state.state

        # For each list in the problem
        for element in start_problem:

            # Needed to not have duplicates
            # Add new element to "current_state"
            if element not in current_state.state:
                n_of_visited_nodes += 1
                state_list = curr_state.copy()
                state_list.append(element)
                number_found = set(element) | current_state.number_found

                temp_state = State(state_list, number_found ,g(state_list, element), h(N, element, number_found))
                
                if temp_state.number_found >= goal:
                    print(f"\nSolution using A* algotithm with N = {N} =>\n\t W = {sum(len(element) for element in temp_state.state)} \n\t N of VISITED NODES = {n_of_visited_nodes}")
                    print(f"Solution {temp_state.state}")
                    return
                
                open_states.append(temp_state)


In [23]:
a_star(14)

Lists 54

Solution using A* algotithm with N = 14 =>
	 W = 16 
	 N of VISITED NODES = 175120
Solution [[10, 11], [1, 6], [13, 5], [0, 12], [9, 3], [2, 3, 4, 7, 8, 11]]


#### Professor's Dijkstra Solution

In [14]:
from gx_utils import PriorityQueue, Counter

def dijkstra(N, all_lists):
    """Vanilla Dijkstra's algorithm"""

    GOAL = set(range(N))
    all_lists = tuple(set(tuple(_) for _ in all_lists))
    frontier = PriorityQueue()
    nodes = 0

    def state_to_set(state):
        return set(sum((e for e in state), start=()))

    def goal_test(state):
        return state_to_set(state) == GOAL

    def possible_steps(state):
        current = state_to_set(state)
        return [l for l in all_lists if not set(l) <= current]

    def w(state):
        cnt = Counter()
        cnt.update(sum((e for e in state), start=()))
        return sum(cnt[c] - 1 for c in cnt if cnt[c] > 1), -sum(cnt[c] == 1 for c in cnt)

    state = tuple()
    while state is not None and not goal_test(state):
        nodes += 1
        for s in possible_steps(state):
            frontier.push((*state, s), p=w((*state, s)))
        state = frontier.pop()

    logging.debug(f"dijkstra: SOLVED! nodes={nodes:,}; w={sum(len(_) for _ in state):,}; iw={w(state)})")
    return state

In [15]:
logging.getLogger().setLevel(logging.DEBUG)

for N in [5, 10, 14]:
    solution = dijkstra(N, problem(N, seed=42))
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={sum(len(_) for _ in solution):,} "
        + f"(bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )

DEBUG:root:dijkstra: SOLVED! nodes=3; w=5; iw=(0, -5))
INFO:root: Solution for N=5: w=5 (bloat=0%)
DEBUG:root:dijkstra: SOLVED! nodes=3; w=10; iw=(0, -10))
INFO:root: Solution for N=10: w=10 (bloat=0%)
DEBUG:root:dijkstra: SOLVED! nodes=469; w=14; iw=(0, -14))
INFO:root: Solution for N=14: w=14 (bloat=0%)
